In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
matplotlib.style.use('seaborn-notebook')
plt.rcParams['figure.figsize'] = (14.0, 11.0)
%load_ext autoreload
%autoreload 2

ImportError: No module named 'matplotlib'

In [2]:
import process as p
import dmc

Using gpu device 0: GeForce GTX 970 (CNMeM is disabled, CuDNN not available)


In [3]:
df = p.processed_data()

In [4]:
start, end, split = pd.Timestamp('2014-1-1'), pd.Timestamp('2014-12-31'), pd.Timestamp('2014-10-1')
df.orderDate = pd.to_datetime(df.orderDate)
mask = (df.orderDate >= start) & (df.orderDate <= end)
df_full = df[mask]
df_train = df_full[df_full.orderDate < split]
df_class = df_full[df_full.orderDate >= split]
len(df_train), len(df_class), len(df_full), len(df)

(947782, 288063, 1235845, 2293170)

In [5]:
X, Y, fts = dmc.transformation.transform_preserving_header(
    df_full, scaler=dmc.normalization.scale_features, binary_target=True)

/usr/local/lib/python3.5/site-packages/pandas/core/generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [6]:
X_train, Y_train = X[:5000], Y[:5000]
X_class, Y_class = X[5000:20000], Y[5000:20000]

In [15]:
res_tree = dmc.evaluation.evaluate_features_leaving_one_out(
    X_train, Y_train, X_class, Y_class, fts, dmc.classifiers.DecisionTree)
res_bayes = dmc.evaluation.evaluate_features_leaving_one_out(
    X_train, Y_train, X_class, Y_class, fts, dmc.classifiers.NaiveBayes)
res_svm = dmc.evaluation.evaluate_features_leaving_one_out(
    X_train, Y_train, X_class, Y_class, fts, dmc.classifiers.SVM)